In [1]:
#!pip install datasets bitsandbytes accelerate peft # Comment it out when not using Colab
USER_ACCESS_TOKEN = "" #Deleted private token

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset

In [5]:
raw_dataset = load_dataset("coai/plantuml_generation", "default", split="train")#.select(range(16))

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", add_eos_token=True, use_fast=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(data):
    return tokenizer(data["text"], truncation=True)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

eval_dataloader = None #No evaluation dataset provided

In [6]:
compute_dtype = getattr(torch, "bfloat16")
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=compute_dtype,
                                bnb_4bit_use_double_quant=True)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", 
                                             trust_remote_code=True, 
                                             quantization_config=bnb_config, 
                                             device_map="auto",
                                             torch_dtype="auto")

In [7]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj","k_proj","v_proj","fc2","fc1"]
)

model = get_peft_model(model, peft_config)

In [8]:
training_arguments = TrainingArguments(
        output_dir="finetuned_phi_15_plantuml_generation",
        push_to_hub=True,
        hub_token=USER_ACCESS_TOKEN,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=12,
        log_level="debug",
        save_steps=100,
        logging_steps=25, 
        learning_rate=1e-4,
        optim='paged_adamw_8bit',
        bf16=True, #change to fp16 if are using an older GPU
        num_train_epochs=3,
        warmup_steps=100,
        lr_scheduler_type="linear",
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Using auto half precision backend
Currently training with a batch size of: 1
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,940
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 12
  Total optimization steps = 483
  Number of trainable parameters = 12,582,912
  0%|          | 0/483 [00:00<?, ?it/s]/home/ertozi/miniconda3/envs/coding-exercises/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current

KeyboardInterrupt: 

: 

In [ ]:
save_directory = "finetuned_phi_15_plantuml_generation"
model.save_pretrained(save_directory, push_to_hub=True, token=USER_ACCESS_TOKEN)
tokenizer.save_pretrained(save_directory, push_to_hub=True, token=USER_ACCESS_TOKEN) 

model.safetensors:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer("Generate a plantuml diagram", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a plantuml diagram for the given number of nodes.
    """
    def generate_node(node: int, parent: int, depth: int) -> str:
        """
        Generate a node for the given node, parent, and depth.
        """
        if depth == 0:
            return f"{node}:{parent}"
        else:
            return f"{node}:{parent}:{generate_node(node, parent, depth-1)}"

    def generate_edges(node: int, parent: int, depth: int) -> List[str]:
        """
        Generate a list of edges for the given node, parent, and depth.
        """
        if depth == 0:
            return [f"{node}:{parent}"]
        else:
            return [f"{node}:{parent}:{generate_edge(node, parent, depth-1
